In [1]:
print('hello world')

hello world


## Data Acquisition

In [9]:
import pandas as pd

users_df = pd.read_csv('/Users/ineshtandon/Documents/GitHub/CF_RecSystem/data/Users.csv')
books_df = pd.read_csv('/Users/ineshtandon/Documents/GitHub/CF_RecSystem/data/Books.csv')
ratings_df = pd.read_csv('/Users/ineshtandon/Documents/GitHub/CF_RecSystem/data/Ratings.csv')

/var/folders/ph/5hgyd1hn7dgcjfc5l9h535tc0000gn/T/ipykernel_74706/3673212882.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv('/Users/ineshtandon/Documents/GitHub/CF_RecSystem/data/Books.csv')


In [2]:
users_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [3]:
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


## Data Processing and creation

In [10]:
from sklearn.preprocessing import LabelEncoder
def preprocess_trainSet(train_df):
    user_encoder = LabelEncoder()
    book_encoder = LabelEncoder()

    train_df['user_idx'] = user_encoder.fit_transform(train_df['User-ID'])
    train_df['book_idx'] = book_encoder.fit_transform(train_df['ISBN'])

    return train_df, user_encoder, book_encoder

def preprocess_testSet(test_df, user_encoder, book_encoder):
    test_df = test_df[
        test_df['User-ID'].isin(user_encoder.classes_) &
        test_df['ISBN'].isin(book_encoder.classes_)
    ].copy()

    # Transform using same encoders
    test_df['user_idx'] = user_encoder.transform(test_df['User-ID'])
    test_df['book_idx'] = book_encoder.transform(test_df['ISBN'])

    return test_df

In [11]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    ratings_df, 
    test_size=0.25, 
    random_state=42, 
    stratify=None
)

train_df, user_encoder, book_encoder = preprocess_trainSet(train_df)
test_df = preprocess_testSet(test_df, user_encoder, book_encoder)

In [12]:
train_df.shape, test_df.shape

((862335, 5), (215830, 5))

In [14]:
from scipy.sparse import csr_matrix
def create_sparse_matrix(train_df):
    n_users = train_df['user_idx'].nunique()
    n_books = train_df['book_idx'].nunique()

    rating_matrix = csr_matrix(
        (train_df['Book-Rating'], (train_df['user_idx'], train_df['book_idx'])),
        shape=(n_users, n_books)
    )

    return rating_matrix


In [15]:
utility_matrix = create_sparse_matrix(train_df)
utility_matrix.shape

(89668, 287246)

## Item-Item Collaborative Filtering

In [16]:
# For item-item collaborative filtering, transposing utility_matrix

ii_utility_matrix = utility_matrix.T.copy()

In [17]:
ii_utility_matrix.shape

(287246, 89668)

### Exploring k means

Cosine similarity on the utility matrix inspite of batch wise or entire matrix is causing memory overflow and crashing the kernel

In [18]:
from sklearn.neighbors import NearestNeighbors

# item_user_matrix is (n_books x n_users) sparse matrix
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=50, n_jobs=-1)
model_knn.fit(ii_utility_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=50)

In [19]:
# Returns distances and indices of top-k similar items
distances, indices = model_knn.kneighbors(ii_utility_matrix)

# similarity = 1 - distance
similarities = 1 - distances


In [27]:
indices[0]

array([     0, 191488, 191495, 191494, 191493, 191492, 191491, 191490,
       191489, 191487, 191462, 191460, 191467, 191466, 191465, 191464,
       191463, 191469, 191461, 191456, 191457, 191455, 191471, 191496,
       191497, 191498, 191499, 191520, 191519, 191518, 191517, 191516,
       191515, 191514, 191513, 191512, 191511, 191510, 191509, 191508,
       191507, 191506, 191505, 191504, 191503, 191502, 191501, 191500,
       191458, 191459])

In [29]:
# predicting function
import numpy as np

def predict_item_item_rating(user_idx, item_idx, rating_matrix, indices, similarities):
    user_ratings = rating_matrix[user_idx, :].toarray().flatten()
    
    # Get top-k similar items to item_idx
    sim_items = indices[item_idx]
    sim_scores = similarities[item_idx]
    
    # Filter to those that user has rated
    rated_mask = user_ratings[sim_items] > 0
    if not np.any(rated_mask):
        return np.nan  # no similar rated items
    
    relevant_ratings = user_ratings[sim_items[rated_mask]]
    relevant_sims = sim_scores[rated_mask]
    
    return np.dot(relevant_ratings, relevant_sims) / np.sum(np.abs(relevant_sims))

In [30]:
from sklearn.metrics import mean_absolute_error

y_true = []
y_pred = []

for _, row in test_df.iterrows():
    u_idx = row['user_idx']
    i_idx = row['book_idx']
    true_rating = row['Book-Rating']
    
    pred_rating = predict_item_item_rating(u_idx, i_idx, utility_matrix, indices, similarities)
    
    if not np.isnan(pred_rating):
        y_true.append(true_rating)
        y_pred.append(pred_rating)

mae = mean_absolute_error(y_true, y_pred)
print(f"MAE (Top-K Item-Item CF): {mae:.4f}")


/var/folders/ph/5hgyd1hn7dgcjfc5l9h535tc0000gn/T/ipykernel_74706/2709213999.py:19: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(relevant_ratings, relevant_sims) / np.sum(np.abs(relevant_sims))


MAE (Top-K Item-Item CF): 3.9702
